In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import numpy as np
from torch.utils.data import random_split
from xgboost.sklearn import XGBClassifier


In [2]:
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',', dtype=np.float32)
    x = data[:, 0:-2]
    y = data[:, [-1]] # after 3 months (-2) and 6 months (-1)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [3]:
# target to fix code


class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Sigmoid())
        self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        #self.layer3 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        self.dropout = nn.Dropout(0.005)
        #self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Tanh())
        self.activation = torch.nn.ELU()
        #self.dropout = nn.Dropout(0.1)
        #self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        #self.dropout = nn.Dropout(0.1)
        #self.activation = torch.nn.ReLU()
        
        
        

        
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.layer2(x)
        #x = self.dropout(x)
        #x = self.layer3(x)
        
        
        
        return x # return model

In [4]:
x_train, y_train = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_training_dataset.csv')
x_test, y_test = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_test_dataset.csv')

In [5]:
y_train

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
      

In [6]:
x_test

tensor([[0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.2000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.3000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 0.7000, 0.6111, 1.0000, 0.1652, 0.4800],
        [0.0042, 0.4179, 0.2535, 0.5000, 0.2500, 0.2727, 0.3986, 0.0000, 0.0000,
         0.2388, 0.6000, 0.5000, 1.0000, 0.1739, 0.7500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 1.0000, 0.7222, 0.0000, 0.1739, 0.0000],
        [0.3208, 0.4467, 0.2676, 0.5000, 0.7500, 0.2727, 0.7489, 0.2812, 0.3559,
         0.1939, 1.0000, 0.7222, 0.0000, 0.3043, 0.7500],
        [0.0084, 0.5389, 0.0000, 1.0000, 0.2500, 0.1818, 0.6514, 0.0000, 0.0735,
         0.3152, 0.4000, 1.0000, 1.0000, 0.2609, 0.0000],
        [0.0168, 0.7176, 0.

In [7]:
x_train.shape

torch.Size([103, 15])

In [8]:


from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler,MaxAbsScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) 


x_train=torch.Tensor(x_train)
x_test=torch.Tensor(x_test)

In [10]:
model = FNN(input_size=15, hidden_size=300, output_size=1)
criterion = nn.BCELoss()
#optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix
optimizer = optim.NAdam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix

In [11]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0457, -0.2415, -0.2450,  ...,  0.2309, -0.2440,  0.0230],
        [ 0.2221, -0.1940,  0.2211,  ...,  0.1426,  0.2460, -0.2182],
        [-0.0154,  0.0284, -0.0134,  ..., -0.2485, -0.0165,  0.0811],
        ...,
        [ 0.0470, -0.0624,  0.0787,  ...,  0.0157,  0.0725,  0.0151],
        [-0.1005, -0.0972,  0.0372,  ...,  0.0595, -0.1881, -0.1344],
        [-0.2064, -0.1518, -0.1361,  ..., -0.2502,  0.2462,  0.0797]],
       requires_grad=True)
Parameter containing:
tensor([-1.2223e-01, -5.9778e-02,  2.4344e-01,  6.9652e-02,  6.5237e-02,
        -1.6894e-01,  2.7020e-02, -1.2587e-01,  2.0444e-01,  2.0584e-01,
         1.2499e-01, -1.8501e-01, -9.9463e-02, -1.9998e-01, -1.4563e-01,
        -1.7368e-01, -1.1310e-01, -8.3769e-03, -1.3717e-01, -8.6341e-02,
         1.4588e-02,  2.6092e-04,  1.3095e-02,  1.6032e-01,  1.7445e-01,
         1.7259e-01,  1.7435e-01, -1.1173e-01,  8.0787e-02,  2.7727e-02,
        -6.9370e-02,  2.3692e-01, -2.0724e-01, -1.6761e-0

In [12]:
# assuming y is torch.Tensor

In [13]:
num_epochs = 1001

for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
        

with torch.no_grad(): 
    train_accuracy = (model(x_train).gt(0.5).float() == y_train).float().mean().item()
    test_accuracy = (model(x_test).gt(0.5).float() == y_test).float().mean().item()
    
    

print(f'\nTrain Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
#Train Accuracy: 0.9709 -> start
#Test Accuracy: 0.9000 -> start

Epoch [0/1001], Loss: 0.7853
Epoch [200/1001], Loss: 0.1922
Epoch [400/1001], Loss: 0.0578
Epoch [600/1001], Loss: 0.0484
Epoch [800/1001], Loss: 0.0338
Epoch [1000/1001], Loss: 0.0232

Train Accuracy: 1.0000
Test Accuracy: 0.9500


In [14]:
model(x_test).gt(0.5)

tensor([[False],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False]])

In [15]:
model(x_test).gt(0.5).float()

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [16]:
y_test

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])